
Tensorflow is a framework for deep learning. It is a symbolic math library, and is also used for machine learning applications such as neural networks. One of the main thing it uses is the Computational Graph. The main thing it helps with is that we can split the graph into many parts and use our hardware across many servers to calculate it.

<div style="text-align: center;">
<img src="Images/ComGraph.png" alt="Alt text describing the image" width="500"/>
</div>


In [91]:
import tensorflow as tf
print(tf.__version__)
tf.compat.v1.disable_eager_execution() # need to disable eager in TF2.
tf.config.run_functions_eagerly(True)

2.13.0


In [92]:
import tensorflow as tf
# Đây là các bước có thể xây dựng một computational graph
x = tf.Variable(3, name="x")
y = tf.Variable(4, name="y")
f = x*x*y + y + 2
# Tuy nhiên, để có thể chạy nó thì cần phải có một session. Session ở đây được định nghĩa là một cách để chạy một computational graph.
with tf.compat.v1.Session() as sess:
    x.initializer.run()
    y.initializer.run()
    result = f.eval()
    print(result)

42


In [93]:
x1 = tf.Variable(5)
print(x1.graph is tf.compat.v1.get_default_graph())
# Ngay khi một node được tạo ra, nó sẽ được thêm vào default graph

True


In [94]:
# Trong trường hợp chúng ta muốn có nhiều graph thì chúng ta có thể tạo ra một graph mới 
graph = tf.Graph() # Graph của chúng ta
with graph.as_default():
    x2 = tf.Variable(10)
print(x2.graph is graph)
print(x2.graph is tf.compat.v1.get_default_graph())
tf.compat.v1.reset_default_graph()
# Ở trong tensorflow nếu như muốn dùng 1 hàm mà nó chỉ có ở ver trước nhưng k có ở ver này thì chúng ta có thể sử dụng hàm compat.v1 và compat.v2 

True
False


In [95]:
# Vòng đời của 1 node trong graph
w = tf.constant(3)
x = w + 2
y = x + 5
z = x * 3
with tf.compat.v1.Session() as _:
    print(y.eval())  # 10
    print(z.eval())  # 15
# TensorFlow sẽ tính toán lại các node mà nó cần để tính toán node mà chúng ta cần. Nó sẽ tự động dò được các biến phụ thuộc. Tuy nhiên, nó sẽ không dùng lại kết quả của các node mà nó đã tính toán trước đó.

10
15


In [96]:
# Nếu như chúng ta không muốn tính toán lại 2 lần kết quả thì chúng ta 
with tf.compat.v1.Session() as _:
    y_val, z_val = _.run([y, z])
    # Tính toán y_val và z_val trong 1 lượt chạy 
    print(y_val)  # 10
    print(z_val)  # 15  

10
15


In [97]:
# Tf ops có thể từ các input trả về các output.Nếu như là biến thì nó gọi là source ops. Input và ouput đều là các multidimensional arrays aka TENSOR. Vì thế chúng ta có cái tên là Tensorflow

import numpy as np
from sklearn.datasets import fetch_california_housing

housing = fetch_california_housing()
m, n = housing.data.shape
print(m, n)

20640 8


In [ ]:
housing_data_plus_bias = np.c_[np.ones((m, 1)), housing.data]

X = tf.constant(housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")
    
XT = tf.transpose(X)
theta = tf.matmul(tf.matmul(tf.linalg.inv(tf.matmul(XT, X)), XT), y)


with tf.compat.v1.Session() as _:
    theta_value = theta.eval()
    print(theta_value)

Chúng ta sẽ sử dụng autodiff thay vì đi tính tay mấy cái đạo hàm. 1 Ví dụ có thể thấy là exp(exp(exp(x))). Hình như là cần tận 9 lần để có thể tính toán được đạo hàm của nó. May cho chúng ta thì autodiff ở trong tf ops có thể giải quyết cho chúng ta điều này. Nó có thể tự động tính cho mình gradient.
```` python 
gradients = tf.gradients(mse, [theta])[0]

Nó sẽ tự động tính gradient dựa theo mse và theta. theta ở đây đang là list các biến mà chúng ta muốn tính gradient theo. 
Tóm lại thứ chúng ta cần là 2 thứ
optimize dùng để : Tối ưu hàm mất mát (loss function) thông qua thuật toán tối ưu hóa nào đó (như Gradient Descent). Trong TensorFlow, bạn thường sử dụng một optimizer như tf.train.GradientDescentOptimizer hoặc các optimizer khác như AdamOptimizer, MomentumOptimizer, v.v.
Các optimizer này có phương thức minimize() giúp tự động tính toán gradient và cập nhật giá trị của các biến theo gradient đó.


training_op dùng để: Đại diện cho thao tác (operation) cập nhật giá trị của các biến trong quá trình huấn luyện. Khi bạn chạy sess.run(training_op), nó sẽ thực hiện một bước tối ưu (ví dụ: một bước Gradient Descent) để cập nhật giá trị của các biến dựa trên gradient của hàm mất mát.
````
 

In [131]:
import tensorflow as tf
n_epochs = 1000
learning_rate = 0.01
tf.compat.v1.disable_eager_execution()
x = tf.Variable(10.0, trainable=True)
# Định nghĩa hàm loss
loss = tf.square(x)

# Định nghĩa optimizer là Gradient Descent
optimizer = tf.compat.v1.train.MomentumOptimizer (learning_rate=learning_rate,
                                                  momentum=0.9)
training_op = optimizer.minimize(loss)

values = []
smallest_loss = float('inf')  # set to infinity initially
best_epoch = -1
best_x_value = None


with tf.compat.v1.Session() as sess:
    sess.run(tf.compat.v1.global_variables_initializer())
    for epoch in range(n_epochs):
        # rất bực ở chỗ là mỗi lần chạy thì nó phải sess.run
        _, loss_value, x_value = sess.run([training_op, loss, x])
        if loss_value < smallest_loss:
            smallest_loss = loss_value
            best_epoch = epoch
            best_x_value = x_value
        values.append((epoch, loss_value, x_value))
        if epoch % 100 == 0:
            print("Epoch: ", epoch, "Loss: ", loss_value, "x_value: ", x_value)

print()
print()
print("Best Epoch: ", best_epoch, "Best x_value: ", best_x_value, "Smallest Loss: ", smallest_loss)

Epoch:  0 Loss:  100.0 x_value:  10.0
Epoch:  100 Loss:  0.0017876907 x_value:  0.04228109
Epoch:  200 Loss:  4.2703396e-10 x_value:  -2.06648e-05
Epoch:  300 Loss:  1.5444532e-12 x_value:  -1.2427603e-06
Epoch:  400 Loss:  4.4233583e-17 x_value:  -6.650833e-09
Epoch:  500 Loss:  3.048242e-23 x_value:  -5.521089e-12
Epoch:  600 Loss:  2.092928e-26 x_value:  1.4466955e-13
Epoch:  700 Loss:  9.697628e-31 x_value:  9.847654e-16
Epoch:  800 Loss:  3.4687296e-36 x_value:  1.8624526e-18
Epoch:  900 Loss:  0.0 x_value:  -1.5366877e-20


Best Epoch:  826 Best x_value:  1.809685e-20 Smallest Loss:  0.0


Feeding data cho thuật toán. Chúng ta có thể dùng placeholder để feed data cho thuật toán. Placeholder là một node mà nó sẽ không tính toán được. Nó chỉ là một node mà chúng ta có thể feed data vào. Chúng ta có thể feed data vào placeholder bằng cách sử dụng feed_dict. Feed_dict là một dictionary mà key là placeholdeBr và value là data mà chúng ta muốn feed vào.

In [133]:
A = tf.compat.v1.placeholder(tf.float32, shape=(None, 3))
B = A + 5

AssertionError: 